### The goal of this notebook is to make knowledge graph data more useable.

### Content of this notebook:

0. Packages installation, imports and connection to neo4j database

1. Quick exploration of the knowledge graph: list of unique nodes and relations

2. Data structuration from the knowledge graph

3. Use case views



## 0. Packages installation, imports and connection to neo4j database

In [1]:
# ! pip install neo4j

In [2]:
from neo4j import GraphDatabase
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

# Define the connection URI and authentication
uri = "bolt://94.230.217.43:7687" #"bolt://localhost:7687" #"bolt://94.230.217.43:7687"  # Update this if you're using a different address   localhost
username = "neo4j"  # Your Neo4j username
password = "lagginhorn" #"lagginhorn"  # Your Neo4j password   newpassword

# Create a Neo4j driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

## 1. Quick exploration of the knowledge graph: list of unique nodes and relations

### Utils

In [3]:
# This functions to get the list of unique node labels contained in the knowledge graph

def get_unique_node_labels():
    with driver.session() as session:
        # Cypher query to get all labels of nodes
        result = session.run("MATCH (n) RETURN DISTINCT labels(n) AS label_list")
        # Extracting and flattening the label lists
        unique_labels = set()
        for record in result:
            unique_labels.update(record["label_list"])
        return unique_labels

# Fetch and print the unique node labels
def get_unique_node_labels_main():
    unique_labels = get_unique_node_labels()
    for label in unique_labels:
        print(label)

In [4]:
# This functions to get the list of unique relationships related to one type of node contained in the knowledge graph

def get_unique_relationships(tx, node_label):
    query = """
    MATCH (b:{})-[r]-()
    RETURN DISTINCT type(r) AS relationship
    """.format(node_label)
    result = tx.run(query)    
    return [record["relationship"] for record in result]

# Main block to run the query
def get_unique_relationships_main(node_label="Bill"):
    with driver.session() as session:
        unique_relationships = session.execute_read(get_unique_relationships, node_label)
        print("Unique relationships related to '{}' nodes:".format(node_label))
        for rel in unique_relationships:
            print(rel)

In [5]:
# This functions to get the list of unique relationships related to one type of node contained in the knowledge graph and the node labels associated to the relationships

def get_unique_relationships_with_nodes(tx, node_label):
    query = """
    MATCH (b:{})-[r]->(n)
    RETURN DISTINCT type(r) AS RelationshipType, labels(n) AS RelatedNodeLabels
    """.format(node_label)
    result = tx.run(query)    
    return [(record["RelationshipType"], record["RelatedNodeLabels"]) for record in result]

# Main block to run the query
def get_unique_relationships_with_nodes_main(node_label="Bill"):
    with driver.session() as session:
        unique_relationships = session.execute_read(get_unique_relationships_with_nodes, node_label)
        print("Unique relationships related to '{}' nodes with related nodes:".format(node_label))
        for rel in unique_relationships:
            print(rel)

In [6]:
# This functions to get an extraction of data of one type of node

def get_node_data(tx, node_label):
    query = """
    MATCH (b:{})
    RETURN b LIMIT 10
    """.format(node_label)
    result = tx.run(query)
    return [record["b"] for record in result]

# Main block to run the query
def get_node_data_main(node_label="Bill"):
    list_nodes = []
    with driver.session() as session:
        examples = session.execute_read(get_node_data, node_label)
        # Output the examples
        for example in examples:
            print(example)
            print("-" * 40)
            list_nodes.append(example)
    return list_nodes

### Examples

In [7]:
get_unique_node_labels_main()

Speech
Postprocessing Necessary
Question
Simple Cooperative
Enactment Draft
Postulate
Unexpected Vote Total
Federal Council Dispatch
Committee Debate
Parliamentary Group
Not Bill Related
Unique Person
Interpellation
Inconsistent Timestamp
Tag
Proposal
Session
Federal Commission
Limited Partnership
Transcript
Commission documents
Partition
Person
Cooperative
Submitted
Department
Ambiguous Person
Discussion
Legislative Period
Petition
URL
Public Foundation
Delegation
Foundation
Chamber Debate
Message/Report not in BBl
General Partnership
Cantonal Initiative
Negotiation booklet
Consultation
Employer
Bill
Corporation
Committee
Federal Response
Public Body
Reasoning
Legislative Proposition
City
Public Institute
Party Class
Canton
Interest Group
Question Time
Debate
Written Response
Vote
Ambiguous Employer
Media Summary
Motion
Location
Party
Organisation
Other Parliamentary Activities
News Report
Chamber
Commission
Popular vote
Parliamentary Initiative
Year
Club
MultiDebate
Limited Liability

In [8]:
get_unique_relationships_main("Person")

Unique relationships related to 'Person' nodes:
BORN_IN
GAVE
PART_OF
ACTING_AS
COSPONSORS
SPONSORS
ACTIVE_DURING
CITIZEN_IN
ELECTED_TO
MEMBER_OF
REPRESENTS
ASSOCIATED_WITH
VOTED
WORKS_FOR
CANDIDATE


In [9]:
get_unique_relationships_with_nodes_main("Person")

Unique relationships related to 'Person' nodes with related nodes:
('BORN_IN', ['City', 'Location'])
('GAVE', ['Transcript', 'Speech'])
('PART_OF', ['Committee'])
('PART_OF', ['Delegation'])
('ACTING_AS', ['Party', 'Organisation'])
('ACTING_AS', ['Committee'])
('ACTING_AS', ['Delegation'])
('COSPONSORS', ['Bill', 'Motion'])
('COSPONSORS', ['Bill', 'Postulate'])
('COSPONSORS', ['Bill', 'Interpellation'])
('COSPONSORS', ['Bill', 'Parliamentary Initiative'])
('SPONSORS', ['Bill', 'Question'])
('SPONSORS', ['Bill', 'Motion'])
('SPONSORS', ['Bill', 'Parliamentary Initiative'])
('SPONSORS', ['Bill', 'Postulate'])
('SPONSORS', ['Bill', 'Interpellation'])
('ACTIVE_DURING', ['Year'])
('CITIZEN_IN', ['City', 'Location'])
('ELECTED_TO', ['Chamber'])
('MEMBER_OF', ['Party', 'Organisation'])
('MEMBER_OF', ['Parliamentary Group'])
('REPRESENTS', ['Canton', 'Location'])
('ASSOCIATED_WITH', ['Organisation', 'Interest Group', 'Postprocessing Necessary', 'Foundation'])
('ASSOCIATED_WITH', ['Organisation

In [10]:
list_nodes = get_node_data_main("Person")

<Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:516582' labels=frozenset({'Unique Person', 'Person'}) properties={'date_birth': neo4j.time.Date(1944, 4, 2), 'uid': 532, 'gender': 'm', 'last_name': 'Stähelin', 'first_name': 'Philipp'}>
----------------------------------------
<Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:516583' labels=frozenset({'Unique Person', 'Person'}) properties={'date_birth': neo4j.time.Date(1867, 11, 29), 'uid': 2484, 'gender': 'm', 'last_name': 'Keller', 'date_death': neo4j.time.Date(1932, 4, 21), 'first_name': 'Gustav'}>
----------------------------------------
<Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:516584' labels=frozenset({'Unique Person', 'Person'}) properties={'date_birth': neo4j.time.Date(1889, 6, 21), 'uid': 1762, 'gender': 'm', 'last_name': 'Celio', 'date_death': neo4j.time.Date(1980, 2, 22), 'first_name': 'Enrico'}>
----------------------------------------
<Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:51

## 2. Data structuration from the knowledge graph

### The goal of this section is to get a dataframe with the following columns:

- Bill
- Cantons
- Persons
- Debate
- Tags

### Utils

In [11]:
# This function to get the data of related nodes that are in relation with a spectific relation

# /!\ Set the LIMIT of returned data
def get_node_data_with_relation_and_node(tx, node_label, relation, node_label_2):
    query = """
    MATCH (b:{})-[r:{}]->(rdata:{})
    RETURN b, rdata LIMIT 100
    """.format(node_label, relation, node_label_2)
    result = tx.run(query)
    return [{node_label: record["b"], node_label_2: record["rdata"]} for record in result]

# Main block to run the query
def get_node2_data_with_relation_and_node_data_main(node_label="Person", relation="SPONSORS", node_label_2="Bill"):
    with driver.session() as session:
        examples = session.execute_read(get_node_data_with_relation_and_node, node_label, relation, node_label_2)
        # Output the examples
        df = pd.DataFrame(columns = [node_label_2, node_label])
        print("Node2 {} data".format(node_label_2), examples[0][node_label_2])
        print("Node1 {} data".format(node_label), examples[0][node_label])
        for example in examples:
            df.loc[len(df)] = [example[node_label_2], example[node_label]]
    return df

In [12]:
# This function to get the data of one related node that are in relation with one specific node (defined by an element id), for a specific relation

def get_one_node_data_with_relation_and_node(tx, node_label, relation, node_label_2, element_id):
    query = """
    MATCH (b:{})-[r:{}]->(rdata:{})
    where elementId(b) = '{}'
    RETURN b, rdata LIMIT 1000
    """.format(node_label, relation, node_label_2, element_id)
    result = tx.run(query)
    return [{node_label: record["b"], node_label_2: record["rdata"]} for record in result]

# Main block to run the query
def get_one_node_data_with_relation_and_node_main(element_id, node_label="Person", relation="REPRESENTS", node_label_2="Canton"):
    with driver.session() as session:
        examples = session.execute_read(get_one_node_data_with_relation_and_node, node_label, relation, node_label_2, element_id)
        # Output the examples
        # print("Node1 {} data".format(node_label), examples[0][node_label])
        # print("Node2 {} data".format(node_label_2), examples[0][node_label_2])
    if examples:
       return examples[0][node_label_2]
    #print("NOT FOUND")
    return "NOT FOUND"

### Run

In [13]:
# Get 10000 Bill with Person Sponsors
df = get_node2_data_with_relation_and_node_data_main(node_label="Person", relation="SPONSORS", node_label_2="Bill")
df["Bill"] = df["Bill"].apply(lambda l : {"element_id": l.element_id, "bill_number": l['bill_number'], "title_fr": l['title_fr'], "title_de": l['title_de'], "urgent": l['urgent']})
df["Person"] = df["Person"].apply(lambda l : {"element_id": l.element_id, "last_name": l['last_name'], "first_name":l["first_name"], "gender": l['gender']})

# Associate Canton to Person
df["Canton"] = df["Person"].progress_apply(lambda l: get_one_node_data_with_relation_and_node_main(l["element_id"]))
df["Canton"] = df["Canton"].apply(lambda l : {"name": l['name'], "abbrev": l['abbrev']})

# Get Tag for each Bill  
df["Tag"] = df["Bill"].progress_apply(lambda l: get_one_node_data_with_relation_and_node_main(l["element_id"], node_label="Bill", relation="ABOUT", node_label_2="Tag"))
df = df[df["Tag"]!="NOT FOUND"]
df["Tag"] = df["Tag"].apply(lambda l : {"name": l['name']})

# Associate Debates
df["Debate"] = df["Bill"].progress_apply(lambda l: get_one_node_data_with_relation_and_node_main(l["element_id"], node_label="Bill", relation="CONTAINS", node_label_2="Debate"))

Node2 Bill data <Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454275' labels=frozenset({'Question Time', 'Bill'}) properties={'title_de': 'Kyoto-Protokoll. Bilanz der bisherigen Politik?', 'bill_number': '09.5595', 'title_fr': 'Protocole de Kyoto. Bilan de la politique menée?', 'urgent': False, 'status': 'Resolved'}>
Node1 Person data <Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:585934' labels=frozenset({'Unique Person', 'Person'}) properties={'date_birth': neo4j.time.Date(1969, 5, 5), 'uid': 3888, 'gender': 'm', 'last_name': 'Hiltpold', 'first_name': 'Hugues'}>


100%|██████████| 94/94 [00:00<00:00, 115.68it/s]


In [14]:
df

,Bill,Person,Canton,Tag,Debate
0,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Genf', 'abbrev': 'GE'}",{'name': 'Umwelt'},"(uid, date_start, date_end, order_stamp)"
1,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Zürich', 'abbrev': 'ZH'}",{'name': 'Medien und Kommunikation'},"(uid, date_start, date_end, order_stamp)"
2,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Aargau', 'abbrev': 'AG'}",{'name': 'Internationale Politik'},"(uid, date_start, date_end, order_stamp)"
3,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Bern', 'abbrev': 'BE'}",{'name': 'Verkehr'},"(uid, date_start, date_end, order_stamp)"
4,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Zürich', 'abbrev': 'ZH'}",{'name': 'Recht Allgemein'},"(uid, date_start, date_end, order_stamp)"
...,...,...,...,...,...
95,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Neuenburg', 'abbrev': 'NE'}",{'name': 'Wirtschaft'},"(uid, date_start, date_end, order_stamp)"
96,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Solothurn', 'abbrev': 'SO'}",{'name': 'Sicherheitspolitik'},"(uid, date_start, date_end, order_stamp)"
97,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Bern', 'abbrev': 'BE'}",{'name': 'Medien und Kommunikation'},"(uid, date_start, date_end, order_stamp)"
98,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Bern', 'abbrev': 'BE'}",{'name': 'Medien und Kommunikation'},"(uid, date_start, date_end, order_stamp)"


## 3. Use case views

### The goal of this section is to build a view that would be useful for data visualisations for use cases

In [15]:
df_uc = df.copy()
df_uc["bill_id"] = df_uc["Bill"].apply(lambda l : l["element_id"])
df_uc["bill_title_fr"] = df_uc["Bill"].apply(lambda l : l["title_fr"])
df_uc["bill_title_de"] = df_uc["Bill"].apply(lambda l : l["title_de"])

df_uc["person_name"] = df_uc["Person"].apply(lambda l : l["last_name"] + " " + l["first_name"])
df_uc["person_gender"] = df_uc["Person"].apply(lambda l : l["gender"])

df_uc["canton_name"] = df_uc["Canton"].apply(lambda l : l["name"])
df_uc["canton_abbrev"] = df_uc["Canton"].apply(lambda l : l["abbrev"])

df_uc["topic"] = df_uc["Tag"].apply(lambda l : l["name"])

df_uc["date_start"] = df_uc["Debate"].apply(lambda l : str(l["date_start"]))

df_uc.drop(columns = ["Bill", "Person", "Canton", "Tag", "Debate"], inplace=True)
df_uc.drop_duplicates(["bill_id"], inplace = True)
df_uc

,bill_id,bill_title_fr,bill_title_de,person_name,person_gender,canton_name,canton_abbrev,topic,date_start
0,4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454275,Protocole de Kyoto. Bilan de la politique menée?,Kyoto-Protokoll. Bilanz der bisherigen Politik?,Hiltpold Hugues,m,Genf,GE,Umwelt,2009-12-07
1,4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454276,Possibilités d'économies pour la SSR,Einsparungspotenzial bei der SRG,Bäumle Martin,m,Zürich,ZH,Medien und Kommunikation,2009-12-07
2,4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454277,Coup d'Etat au Honduras,Putsch in Honduras,Müller Geri,m,Aargau,AG,Internationale Politik,2009-12-07
3,4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454278,Premières chutes de neige et gestion de la cir...,Wintereinbruch und Verkehrsmanagement auf der A2,Brönnimann Andreas,m,Bern,BE,Verkehr,2009-12-07
4,4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454279,Interdiction d'exercer une profession pour les...,Berufsverbot für Frauen,Kaufmann Hans,m,Zürich,ZH,Recht Allgemein,2009-12-07
...,...,...,...,...,...,...,...,...,...
95,4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454370,"Ordonnance d'application du ""Cassis de Dijon""....",Verordnung zum Cassis-de-Dijon-Prinzip. Eine B...,Favre Laurent,m,Neuenburg,NE,Wirtschaft,2009-12-07
96,4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454371,Découpage des chars de grenadiers M113,Verschrottung von Schützenpanzern M113,Borer Roland F.,m,Solothurn,SO,Sicherheitspolitik,2009-12-07
97,4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454372,Job Fashion,Job Fashion,Wandfluh Hansruedi,m,Bern,BE,Medien und Kommunikation,2009-12-07
98,4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454373,Fusion Orange et Sunrise. Le marché des téléco...,Fusion Orange und Sunrise. Spielt der Wettbewe...,von Graffenried Alec,m,Bern,BE,Medien und Kommunikation,2009-12-07


In [16]:
df_uc.to_csv("/home/jovyan/work/5-2-lagginhorn/use_case_views/uc3_94.csv")

## Appendix - change neo4j password

In [4]:
from neo4j import GraphDatabase

# Define the connection URI and authentication with the expired password
uri = "bolt://localhost:7687"
username = "neo4j"  # Your Neo4j username
current_password = "neo4j"  # The current password you provided
new_password = "newpassword"  # Set the new password

# Create a Neo4j driver instance
driver = GraphDatabase.driver(uri, auth=(username, current_password))

# Function to change the password
def change_password(current_password, new_password):
    try:
        with driver.session(database="system") as session:  # Connect to the system database
            # Cypher query to change the password
            query = f"ALTER CURRENT USER SET PASSWORD FROM '{current_password}' TO '{new_password}'"
            session.run(query)
            print("Password changed successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.close()

# Change the password by executing the query
change_password(current_password, new_password)


Password changed successfully.
